In [1]:
import random
import re
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import itemgetter
import sys
from mnist import MNIST
mndata = MNIST('./2')

In [2]:
pokoleniya=10
population_size=10
chromosome_bodyoncetoldme=784 # длина хромосомы
selection_count=50 # сколько особей выбирается в селекции
mutation_kek=0.05 # шанс мутации, от 0 до 1
log=False # отображение данных после каждой операции

a=1 # скорость обучения
max_epochs=10 # эпохи обучения

In [ ]:
images_train, labels_train = mndata.load_training()
train_fist=mndata.process_images_to_numpy(images_train)
train_ass=mndata.process_labels(labels_train)

In [3]:
def f(v): # функция активации
    if(v>0): return 1
    else: return 0

activ=np.vectorize(f)

In [27]:
def predict(vector):
    result=0
    for i in range(10):
        if(vector[i]==1): result=i
    return result

# функция приспособленности
def prisposobl(w):
    w=np.array(w)
    x=train_fist
    y=train_ass
    y1=x@w
    y1=activ(y1)
    right=0
    for i in range(len(x)):
        if(predict(y1[i])==y[i]): right+=1

    accyracy=right/10000
    return accyracy # сколько правильных ответов



In [26]:
def generate_population(size):
    result=[]
    for kek in range(size):
        w=np.random.rand(chromosome_bodyoncetoldme,1).tolist()
        result.append(w)
    return result

print(generate_population(5)[1])

[[0.5477405557762564], [0.15791604734959652], [0.06046697447318883], [0.8454262928323061], [0.8180518890977034], [0.3494039255838446], [0.012778200115543537], [0.041061840746593914], [0.40814523582368734], [0.9634161895486758], [0.3194523588124818], [0.9114469514286412], [0.6755567665892197], [0.8385766608613634], [0.7025322571332281], [0.9653145750054636], [0.21861254195297286], [0.9375197743878811], [0.37921391164224105], [0.8481302078584786], [0.4846090355163555], [0.7353995756647727], [0.18697232567022104], [0.07007084349744308], [0.7712195861381264], [0.9832771352015213], [0.7175918772391239], [0.30423289185831615], [0.6784776331463707], [0.7086005693907141], [0.21553594766019002], [0.3565125428355559], [0.4858098170920554], [0.4422149452323998], [0.8855355636766028], [0.48620201508131544], [0.33801227630802333], [0.3854078162621819], [0.5380711374594495], [0.7798803064552373], [0.4172329383999608], [0.39710356871052277], [0.5820704137692778], [0.6718660579512085], [0.914832544544

In [ ]:
def print_population(data):
    df=pd.DataFrame(data)
    print(df)

# лучший маршрут в популяции
def population_best_result(data):
    result=0
    best_shrek=[]
    for i in data:
        shrek=prisposobl(i)
        if(shrek>result): 
            result=shrek
            best_shrek=i
    return (result, best_shrek)

# худший маршрут в популяции
def population_max_result(data):
    penis=[]
    for i in data:
        penis.append((prisposobl(i), i))
    max_shrek=sorted(penis, key=itemgetter(0), reverse=True)[0]

    return (max_shrek)

# средний маршрут в популяции
def population_average_result(data):
    penis=[]
    for i in data:
        penis.append((prisposobl(i)))
    result=np.mean(penis)

    return result

# sex=generate_population(5)
# print(population_best_result(sex))
# print(population_max_result(sex))
# print(population_average_result(sex))

In [ ]:

def roulette_selection(data, sel_count):
    pool=[]
    sectors=[]
    wheel=[]
    f_sum=0

    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        f_sum+=f
    for chromosome in data:
        f=1/(1+prisposobl(chromosome))
        v=f/f_sum*100
        sectors.append((v, chromosome))
    # sectors=sorted(sectors, key=itemgetter(0))
    point=0
    for i in range(0,len(sectors)-1):
        wheel.append((sectors[i][1], (point, point+sectors[i][0])))
        point+=sectors[i][0]
        if(i==len(sectors)-2): 
            wheel.append((sectors[i][1], (point, 100)))
    # print(wheel)
    for i in range(sel_count):
        choice=random.randint(0,100)
        for j in wheel:
            # if((choice>=j[1][0]) and (choice<=j[1][1]) and (j[0] not in pool)): 
            if((choice>=j[1][0]) and (choice<=j[1][1])): 
                pool.append(j[0])
                break
    
    return pool

roulette_selection(generate_population(10), 5)

In [23]:

def simple_selection(data, sel_count):
    pool=[]
    adaptation=[]
    for chromosome in data:
        adaptation.append((prisposobl(chromosome), chromosome))
    adaptation=sorted(adaptation, key=itemgetter(0))
    for i in range(0, sel_count):
        pool.append(adaptation[i][1])

    return pool

simple_selection(generate_population(10), 5)

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:

def permutation_crossover(parent1, parent2, chromo_length):
    chromo_length-=2
    par1_start=parent1[0]
    par2_start=parent2[0]
    par1_end=parent1[len(parent1)-1]
    par2_end=parent2[len(parent1)-1]
    parent1=parent1[1:len(parent1)-1]
    parent2=parent2[1:len(parent2)-1]

    pos1=random.randint(1, chromo_length//2-1)
    pos2=random.randint(pos1+1, chromo_length-1)
    middle=parent1[pos1:pos2+1]
    shrek=[]
    for i in parent2:
        if not (i in middle): shrek.append(i)
    child1=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child1.append(shrek.pop(0))
        elif(i==pos1): child1+=middle
        elif(i==pos2): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child1.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child1.append(parent2[i])
    
    middle=parent2[pos1:pos2+1]
    shrek=[]
    for i in parent1:
        if not (i in middle): shrek.append(i)
    child2=[]
    for i in range(0, chromo_length-1):
        if(i<pos1): child2.append(shrek.pop(0))
        elif(i==pos1): child2+=middle
        elif(i==pos2): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)>0): child2.append(shrek.pop(0))
        elif(i>pos2) and (len(shrek)==0): child2.append(parent1[i])

    child1.insert(0, par1_start)
    child1.append(par1_end)
    child2.insert(0, par2_start)
    child2.append(par2_end)
    return child1, child2

permutation_crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1], 8)

In [ ]:

def two_point_crossover(parent1, parent2, chromo_length):
    chromo_length-=2
    par1_start=parent1[0]
    par2_start=parent2[0]
    par1_end=parent1[len(parent1)-1]
    par2_end=parent2[len(parent1)-1]
    parent1=parent1[1:len(parent1)-1]
    parent2=parent2[1:len(parent2)-1]

    pos1=random.randint(1, chromo_length//2-1)
    pos2=random.randint(pos1+1, chromo_length-1)

    child1=parent1[0:pos1]+parent2[pos1-1:pos2]+parent1[pos2+1:chromo_length]
    child2=parent2[0:pos1]+parent1[pos1-1:pos2]+parent2[pos2+1:chromo_length]

    child1.insert(0, par1_start)
    child1.append(par1_end)
    child2.insert(0, par2_start)
    child2.append(par2_end)
    return child1, child2

two_point_crossover([1,2,3,4,5,6,7,8], [8,7,6,5,4,3,2,1], 8)

In [147]:
def mutation(chromosome, chrom_length):
    result=chromosome
    choice=random.random()

    if(choice-mutation_kek<=0):
        pos1=random.randint(2, chrom_length//2-1)
        pos2=random.randint(pos1+1, chrom_length-2)
        shrek=result[pos1:pos2+1]
        for i in range(len(shrek)):
            shrek[i]=random.random()
        result[pos1:pos2+1]=shrek

    return result

mutation([1,2,3,4,5,6,7,8], 8)

[1, 2, 3, 4, 5, 6, 7, 8]

## обучение

In [13]:
# train_filter = []
# for element in train_ass:
#   if (element==0)|(element==1):
#     train_filter.append(True)
#   else:
#     train_filter.append(False)

# x=np.around(train_fist/255)[train_filter] # нейроны
# y=np.array(train_ass)[train_filter]

# print(y)

[0 1 1 ... 1 0 1]


In [6]:
# w=np.random.rand(784,1) # веса
# print(w) # 784x1

[[7.36041220e-02]
 [7.58061235e-01]
 [6.17292184e-01]
 [9.99529101e-01]
 [8.63506429e-01]
 [5.55739202e-01]
 [7.22609822e-01]
 [8.18601730e-01]
 [5.73441566e-01]
 [4.50924170e-01]
 [3.98076790e-02]
 [3.82907122e-02]
 [8.45530070e-01]
 [4.40037799e-01]
 [6.12708692e-01]
 [3.97405843e-01]
 [8.33233644e-01]
 [2.91787100e-01]
 [7.88255458e-01]
 [5.18775963e-01]
 [2.11089236e-01]
 [6.39512772e-02]
 [8.32169722e-02]
 [5.02180604e-01]
 [8.10679179e-01]
 [2.12695441e-01]
 [5.84216572e-02]
 [3.59328082e-01]
 [6.59308320e-01]
 [9.77641487e-01]
 [6.95386724e-01]
 [2.76362047e-01]
 [5.92867660e-01]
 [7.18137659e-01]
 [7.45424784e-01]
 [7.79033007e-02]
 [4.56939209e-01]
 [3.24557603e-01]
 [2.49325439e-02]
 [1.73552249e-01]
 [1.34262815e-02]
 [2.44472688e-01]
 [7.90549448e-01]
 [4.97804162e-01]
 [3.07299626e-01]
 [4.77386245e-01]
 [8.33693000e-01]
 [8.68397666e-03]
 [5.56581182e-01]
 [9.98846806e-01]
 [9.70200505e-03]
 [2.17296377e-01]
 [4.57425784e-01]
 [5.94205896e-01]
 [4.76396583e-01]
 [3.766137

In [11]:
# temp=np.eye(2)
# for epoch in range(max_epochs):
#     for i in range(len(x)):
#         y1=x[i]@w
#         y1=activ(y1)
#         ei=temp[y[i]]-y1
#         Di=a*np.dot(x[i].reshape((784,1)), ei.reshape((1,2))) # 784x1
#         w=w+Di

In [12]:
# print(w)
# y1=x@w # 60000x10
# y1=activ(y1)
# print(y1) # выходные значения

[[0.07360412 0.07360412]
 [0.75806124 0.75806124]
 [0.61729218 0.61729218]
 ...
 [0.63293786 0.63293786]
 [0.67618449 0.67618449]
 [0.09347683 0.09347683]]
[[1 0]
 [0 1]
 [0 1]
 ...
 [0 1]
 [1 0]
 [0 1]]


In [ ]:
average_results=[]
max_results=[]
min_results=[]
best=[]

elite_size=10

if(log):
    current_population=generate_population(population_size)
    print("первая популяция создана:")
    print_population(current_population)
    print("Лучшее значение приспособленности:")
    print(population_best_result(current_population))

    for pok in range(pokoleniya):
        print(f"Поколение {pok+1}")
        if(elite_size>0): 
            elite=simple_selection(current_population, elite_size)
        current_population=roulette_selection(current_population, selection_count) # Здесь менять селекцию
        print("селекция произведена")
        print("результат селекции:")
        print_population(current_population)
        print("Лучшее значение приспособленности после селекции:")
        print(population_best_result(current_population))
        
        for i in range((population_size-selection_count-elite_size)//2):
            parent1=random.randint(0, selection_count-1)
            parent2=random.randint(0, selection_count-1)
            while(parent2==parent1): parent2=random.randint(0, selection_count-1)
            children=permutation_crossover(current_population[parent1], current_population[parent2], chromosome_bodyoncetoldme) # Здесь менять скрещивание
            current_population.append(children[0])
            current_population.append(children[1])
        if(elite_size>0): 
            current_population+=elite
        if(len(current_population)<population_size): 
            current_population.append(generate_population(1)[0])
        print("скрещивание произведено")
        print("результат скрещивания:")
        print_population(current_population)
        print("Лучшее значение приспособленности после скрещивания:")
        print(population_best_result(current_population))

        temp=current_population
        for i_chel in range(0, len(current_population)-1):
            temp[i_chel]=mutation(current_population[i_chel], chromosome_bodyoncetoldme)
        current_population=temp
        print("мутация произведена")
        print("результат мутации:")
        print_population(current_population)
        print("Лучшее значение приспособленности после мутации:")
        sas1=population_best_result(current_population)
        print(sas1)
        min_results.append(sas1[0])
        max_results.append(population_max_result(current_population)[0])
        average_results.append(population_average_result(current_population))
        best=sas1[1]

        print(f"Конец {pok+1} поколения")

In [ ]:
if(not log):
    current_population=generate_population(population_size)

    for pok in range(pokoleniya):
        if(elite_size>0): 
            elite=simple_selection(current_population, elite_size)
        current_population=roulette_selection(current_population, selection_count) # Здесь менять селекцию
        
        for i in range((population_size-selection_count-elite_size)//2):
            parent1=random.randint(0, selection_count-1)
            parent2=random.randint(0, selection_count-1)
            while(parent2==parent1): parent2=random.randint(0, selection_count-1)
            children=permutation_crossover(current_population[parent1], current_population[parent2], chromosome_bodyoncetoldme) # Здесь менять скрещивание
            current_population.append(children[0])
            current_population.append(children[1])

        temp=current_population
        for i_chel in range(0, len(current_population)-1):
            temp[i_chel]=mutation(current_population[i_chel], chromosome_bodyoncetoldme)
        current_population=temp

        if(elite_size>0): 
            current_population+=elite
        if(len(current_population)<population_size): 
            current_population.append(generate_population(1)[0])

        sas1=population_best_result(current_population)
        min_results.append(sas1[0])
        max_results.append(population_max_result(current_population)[0])
        average_results.append(population_average_result(current_population))
        best=sas1[1]
    print(f"Лучший маршрут в конце {pokoleniya} поколения:")
    print(min_results[pokoleniya-1], best)

In [ ]:
# график приспособленности по поколениям
def draw_progress(average_results, max_results, min_results, pok_count):
    x=range(pok_count)
    y1=average_results
    y2=max_results
    y3=min_results
    plt.plot(x,y1)
    plt.plot(x,y2)
    plt.plot(x,y3)
    plt.show()

draw_progress(average_results, max_results, min_results, pokoleniya)